# Mergin Maps Projects Management

Mergin Maps API allows you to manage your projects in a simple and effective way.

First let's install mergin maps client (if not installed yet)

In [ ]:
!pip install mergin-client

Login to Mergin Maps using your an existing user

In [ ]:
# Use here your login username and password
LOGIN=""
PASSW=""

import mergin

client = mergin.MerginClient(login=LOGIN, password=PASSW)

Let's create a new project (empty) on an existing workspace (fill `WORKSPACE` with an existing workspace)

In [ ]:
WORKSPACE=""

client.create_project(project_name='prague-demo', namespace=WORKSPACE, is_public=False)

Download locally your newly created Mergin Maps project to add some content

In [ ]:
client.download_project(project_path='{workspace}/prague-demo'.format(workspace=WORKSPACE) , directory='/tmp/demo-prague')

Let's add some random points over Prague city and save it as geopackage. Push the changes on our demo project.

In [ ]:
# This is needed to compute the geopackage
!pip install pandas geopandas 

In [ ]:
import geopandas
from shapely.geometry import Point
import random
import time # For potential delays if making many requests, though geocode handles some.

def create_random_pois_in_city_gpkg(city_name, num_pois=100, output_gpkg_path="city_random_pois.gpkg", layer_name="random_pois"):
    """
    Generates a specified number of random Points of Interest (POIs) within the
    boundaries of a given city and saves them to a GeoPackage file.

    Args:
        city_name (str): The name of the city and country (e.g., "Prague, Czech Republic").
        num_pois (int): The number of random POIs to generate.
        output_gpkg_path (str): The file path for the output GeoPackage.
        layer_name (str): The name for the layer within the GeoPackage.
    """
    try:
        # Step 1: Geocode the city to get its boundary
        print(f"Attempting to geocode '{city_name}' to fetch its boundary...")
        # Nominatim requires a user_agent. geopandas.tools.geocode passes it to geopy.
        # If you encounter issues, ensure geopy is installed and up-to-date.
        # A unique user_agent is good practice for repeated use.
        city_gdf = geopandas.tools.geocode(city_name, provider="nominatim", user_agent=f"random_poi_generator_{random.randint(1000,9999)}")

        if city_gdf.empty:
            print(f"Error: Could not geocode '{city_name}'. Please check the city name or your internet connection.")
            return

        # Assuming the first result is the correct boundary for the city
        city_boundary_polygon = city_gdf.geometry.iloc[0]
        print(f"Successfully obtained boundary for '{city_name}'. CRS: {city_gdf.crs}")

        # Step 2: Get the bounding box of the city to generate random points
        min_lon, min_lat, max_lon, max_lat = city_boundary_polygon.bounds

        generated_points = []
        poi_attributes = {'id': [], 'name': [], 'category': []}

        print(f"Generating {num_pois} random POIs within '{city_name}'...")

        # Safety counter to prevent potential infinite loops for complex geometries
        max_attempts_per_poi = 1000
        total_attempts = 0

        while len(generated_points) < num_pois:
            if total_attempts > num_pois * max_attempts_per_poi:
                print(f"Warning: Exceeded maximum attempts. Generated {len(generated_points)} out of {num_pois} POIs.")
                break

            # Generate a random point within the bounding box
            random_longitude = random.uniform(min_lon, max_lon)
            random_latitude = random.uniform(min_lat, max_lat)
            candidate_point = Point(random_longitude, random_latitude)

            # Check if the generated point is actually within the city's polygon (not just its bounding box)
            if candidate_point.within(city_boundary_polygon):
                generated_points.append(candidate_point)
                poi_id = len(generated_points)
                poi_attributes['id'].append(poi_id)
                poi_attributes['name'].append(f"POI_{city_name.split(',')[0].replace(' ','_')}_{poi_id}")
                poi_attributes['category'].append(random.choice(["Attraction", "Restaurant", "Shop", "Park", "Service"]))

                if len(generated_points) % 10 == 0:
                    print(f"Generated {len(generated_points)}/{num_pois} POIs...")

            total_attempts += 1

        if not generated_points:
            print("No POIs could be generated within the city boundary. Exiting.")
            return

        # Step 3: Create a GeoDataFrame from the generated points and attributes
        # The CRS of the points should match the CRS of the geocoded city boundary
        pois_gdf = geopandas.GeoDataFrame(poi_attributes, geometry=generated_points, crs=city_gdf.crs)
        print(f"Created GeoDataFrame with {len(pois_gdf)} POIs.")

        # Step 4: Save the GeoDataFrame to a GeoPackage file
        print(f"Saving POIs to '{output_gpkg_path}' in layer '{layer_name}'...")
        pois_gdf.to_file(output_gpkg_path, layer=layer_name, driver="GPKG")
        print(f"Successfully created GeoPackage: '{output_gpkg_path}'")

    except ImportError:
        print("Error: One or more required libraries (geopandas, shapely, geopy) are not installed.")
        print("Please install them using: pip install geopandas shapely geopy")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        print("Please ensure you have an active internet connection for geocoding.")

if __name__ == "__main__":
    # Define the city and parameters
    target_city = "Prague, Czech Republic"
    number_of_pois_to_generate = 500
    output_file_name = "/tmp/demo-prague/prague_random_points.gpkg"
    output_layer_name = "prague_pois"

    create_random_pois_in_city_gpkg(
        city_name=target_city,
        num_pois=number_of_pois_to_generate,
        output_gpkg_path=output_file_name,
        layer_name=output_layer_name
    )

In [ ]:
client.push_project(directory='/tmp/demo-prague')

Check current project status

In [ ]:
client.project_versions(project_path='{workspace}/prague-demo'.format(workspace=WORKSPACE))  

Let's say, now your project is on a 'template' state and you want to create many projects from here. Simply clone the project using the API!

In [ ]:
client.clone_project(source_project_path='{workspace}/prague-demo'.format(workspace=WORKSPACE), cloned_project_name='{workspace}/prague-demo1'.format(workspace=WORKSPACE))
client.clone_project(source_project_path='{workspace}/prague-demo'.format(workspace=WORKSPACE), cloned_project_name='{workspace}/prague-demo2'.format(workspace=WORKSPACE))

Let's delete one of the cloned projects. 

NOTE: using `delete_project_now` will bypass the default value `DELETED_PROJECT_EXPIRATION`. See: https://merginmaps.com/docs/server/environment/#data-synchronisation-and-management

In [ ]:
client.delete_project_now(project_path='{workspace}/prague-demo1'.format(workspace=WORKSPACE))